In [ ]:
import os
import re
import sys
import numpy as np
import pandas as pd
import seaborn as sns
import ptitprince as pt
from scipy.io import loadmat
import matplotlib.pyplot as plt


def plot_raincloud(overlaps: np.ndarray):
    data = {r'$|\varphi_{' + f'{i+1}' + r'}\rangle$': overlaps[:, i] for i in range(num_phi)}
    df = pd.DataFrame(data)
    pt.half_violinplot(data=df, scale='count', width=0.6, inner=None, linewidth=1.2, palette=colors)
    sns.stripplot(data=df, jitter=True, palette=colors, size=1)
    plt.yticks(np.linspace(0, 1, 11))
    plt.ylabel('Overleaps')
    plt.xlim(-0.7, 3.5)

    # plt.savefig(f'{path_img}.pdf', bbox_inches='tight')
    # plt.savefig(f'{path_img}.png', dpi=400, bbox_inches='tight')
    # plt.savefig(f'./img/{name[10:25]}.png', dpi=400, bbox_inches='tight')
    plt.show()


date = '20241210_06'
path_img = './img/overlaps_raincloud'
path_wsl = r'\\wsl.localhost\Fedora\home\ghost\QuditVGON\mats'
path = './mats' if 'linux' in sys.platform else path_wsl
colors = ['#74B816', '#339AF0', '#FD7E14', '#F03E3E']
for name in sorted(os.listdir(path), reverse=True):
    match = re.search(r'(VGON_nqd\d+_\d{8}_\d{6}).mat', name)
    if match and date in name:
        load = loadmat(f'{path}/{name}')
        energy = load['energy'].item()
        n_qudits = load['n_qudits'].item()
        if 'n_train' in load:
            info_str = load['n_train'].item()
        elif 'n_iter' in load:
            info_str = load['n_iter'].item()
        if 'fidelity_max' in load:
            fidelity_max = load['fidelity_max'].item()
        if 'overlaps' in load:
            if energy < -3.99 and fidelity_max < 0.98:
                overlaps = load['overlaps']
                num_phi = overlaps.shape[1]
                if 'count' in load:
                    info_str += ', ' + load['count'].item()
                else:
                    info_str += ', ' + str(overlaps.shape[0])
                print(f'{name}, Energy: {energy:.8f}, Fidelity: {fidelity_max:.8f}, {info_str}')
                plot_raincloud(overlaps)
        else:
            print(f'{name}, Energy: {energy:.8f}, Fidelity: {fidelity_max:.8f}, {info_str}, No overlaps')

In [ ]:
import os
import re
import sys
import numpy as np
import pandas as pd
import seaborn as sns
import ptitprince as pt
from scipy.io import loadmat
import matplotlib.pyplot as plt


def plot_raincloud(overlaps: np.ndarray, name: str, ind: int):
    data = {r'$|\varphi_{' + f'{i+1}' + r'}\rangle$': overlaps[:, i] for i in range(num_phi)}
    i, j = ind // 6, ind % 6
    df = pd.DataFrame(data)
    pt.half_violinplot(data=df, scale='count', width=0.6, inner=None, linewidth=1.2, palette=colors, ax=axs[i][j])
    sns.stripplot(data=df, jitter=True, palette=colors, size=1, ax=axs[i][j])
    axs[i][j].set_xlabel(name[10:25])
    axs[i][j].set_xlim(-0.7, 3.5)
    axs[i][j].set_xticks([])
    axs[i][j].set_yticks(np.linspace(0, 1, 11))


ind = 0
fig, axs = plt.subplots(5, 6, figsize=(18, 15))
date = '202412'
path_img = './img/overlaps_raincloud'
path_wsl = r'\\wsl.localhost\Fedora\home\ghost\QuditVGON\mats'
path = './mats' if 'linux' in sys.platform else path_wsl
colors = ['#74B816', '#339AF0', '#FD7E14', '#F03E3E']
for name in sorted(os.listdir(path), reverse=True):
    match = re.search(r'(VGON_nqd\d+_\d{8}_\d{6}).mat', name)
    if match and date in name:
        load = loadmat(f'{path}/{name}')
        energy = load['energy'].item()
        n_qudits = load['n_qudits'].item()
        if 'n_train' in load:
            info_str = str(load['n_train'].item())
        elif 'n_iter' in load:
            info_str = str(load['n_iter'].item())
        if 'fidelity_max' in load:
            fidelity_max = load['fidelity_max'].item()
        if 'overlaps' in load:
            if energy < -3.99 and fidelity_max < 0.98:
                overlaps = load['overlaps']
                num_phi = overlaps.shape[1]
                if 'count' in load:
                    info_str += f', {load['count'].item()}'
                else:
                    info_str += f', {overlaps.shape[0]}'
                plot_raincloud(overlaps, name, ind)
                ind += 1
        else:
            print(f'{name}, Energy: {energy:.8f}, Fidelity: {fidelity_max:.8f}, {info_str}, No overlaps')

In [ ]:
import sys
import numpy as np
import multiprocessing
from scipy.io import loadmat
import matplotlib.pyplot as plt


def plot_stack(overlaps_sorted):
    psi = [r'$|\psi_{' + f'{i+1}' + r'}\rangle$' for i in range(num_psi)]
    phi = [r'$|\varphi_{' + f'{i+1}' + r'}\rangle$' for i in range(num_phi)]
    for ind in range(num_phi)[::-1]:
        bottom = overlaps_sorted[:, ind + 1:].sum(axis=1)
        plt.bar(psi, overlaps_sorted[:, ind], bottom=bottom, color=colors[ind], alpha=0.8, label=phi[ind])
    handles, labels = plt.gca().get_legend_handles_labels()
    plt.legend(handles=reversed(handles), labels=reversed(labels), loc='lower right')
    plt.yticks(np.linspace(0, 1, 11))
    plt.ylabel('Overlaps')

    # plt.savefig(f'{path_img}.pdf', bbox_inches='tight')
    # plt.savefig(f'{path_img}.png', dpi=400, bbox_inches='tight')
    plt.show()


date = '20241127_033736'
path_img = './img/overlaps_stack'
path_wsl = r'\\wsl.localhost\Fedora\home\ghost\QuditVGON\mats'
path = './mats' if 'linux' in sys.platform else path_wsl
colors = ['#c0eb75', '#a5d8ff', '#ffc078', '#ff8787']
load = loadmat(f'{path}/VGON_nqd7_{date}.mat')
overlaps = load['overlaps']
num_phi = overlaps.shape[1]
num_psi = 10
# 20241127_033736
# 743 0.05 0.05 0.02 (10, 4)
# 884 0.05 0.05 0.02 (10, 4)
# 977 0.05 0.1 0.02 (10, 4)
np.random.seed(743)
overlaps_random = np.random.permutation(overlaps)
overlaps_selected = np.empty([0, num_phi])
for x in overlaps_random:
    if overlaps_selected.any():
        if np.min(x) > 0.05 and np.abs(x - overlaps_selected).min() > 0.02:
            overlaps_selected = np.vstack((overlaps_selected, x))
    else:
        if np.min(x) > 0.05:
            overlaps_selected = x
if overlaps_selected.shape[0] >= 10:
    overlaps_selected = overlaps_selected[0:10]
    overlaps_sorted = overlaps_selected[np.argsort(overlaps_selected[:, 3])]
    plot_stack(overlaps_sorted)
    # print(overlaps_sorted)

In [ ]:
def running(i):
    np.random.seed(i)
    overlaps_rand_perm = np.random.permutation(overlaps)
    for j in np.linspace(0.05, 0.1, 6):
        for k in np.linspace(0.05, 0.1, 6):
            overlaps_selected = np.empty([0, num_phi])
            for x in overlaps_rand_perm:
                if overlaps_selected.any():
                    if np.min(x) > j and np.abs(x - overlaps_selected).min() > tol:
                        overlaps_selected = np.vstack((overlaps_selected, x))
                else:
                    if np.min(x) > k:
                        overlaps_selected = x
            if overlaps_selected.shape[0] >= 10:
                print(i, j, k, tol, overlaps_selected.shape)
                overlaps_selected = overlaps_selected[0:10]
                overlaps_sorted = overlaps_selected[np.argsort(overlaps_selected[:, 3])]
                # print(overlaps_sorted)
                plot_stack(overlaps_sorted)


tol = 0.02
pool = multiprocessing.Pool(32)
pool.map_async(running, range(1000))
pool.close()
pool.join()

```python
seaborn version: 0.11.0
ptitprince version: 0.2.7

.../site-packages/seaborn/axisgrid.py
677  plot_args = [v for k, v in plot_data.items()]  # zyz: replace .iteritems() with .items()

.../site-packages/seaborn/categorical.py
81  iter_data = plot_data.items()  # zyz: replace .iteritems() with .items()

.../site-packages/seaborn/cm.py
1  import matplotlib as mpl
...
1583  # mpl_cm.register_cmap(_name, _cmap)
1584  # mpl_cm.register_cmap(_name + "_r", _cmap_r)
1585
1586  # zyz: replace matplotlib.cm.register with matplotlib.colormaps.register
1587  mpl.colormaps.register(name=_name, cmap=_cmap)
1588  mpl.colormaps.register(name=_name + "_r", cmap=_cmap_r)
```